In [13]:
# Import the libraries

import math
import pandas_datareader as pd_r
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# Set the start and end dates
start_date = '2012-01-01'
end_date = '2024-02-09'

# Fetch the data
df = yf.download('GBPUSD=X', start=start_date, end=end_date)

# Show the data
df

In [ ]:
#Get shape of data

df.shape

In [ ]:
# Visualize the closing price

plt.figure(figsize=(16,8))
plt.title('GBPUSD close price history')
plt.plot(df['Close'])
plt.xlabel('Date',fontsize = 18)
plt.ylabel('Close Value GBPUSD',fontsize = 18)
plt.show()

In [ ]:
# 'Close' data frame
data = df.filter(['Close'])
# Dataframe to numpy array
dataset = data.values

# Get number of rows to train

training_data_len = math.ceil(len(dataset)*0.8)

training_data_len


In [ ]:
# Data Scaling
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data


In [ ]:
# create training dataset (Scaled)

train_data = scaled_data[0:training_data_len,:]
# Split the train_data into x_train and y_train
x_train = []
y_train = []

for i in range (60,len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<=61:
        print(x_train)
        print(y_train)
        print()


In [ ]:
# Convert x_train and y_train to numpy array

x_train,y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Reshape the data for LSTM
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1],1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
# Complie the model
model.compile(optimizer = 'adam', loss = 'mean_squared_error' )

In [ ]:
#Train the model
model.fit(x_train, y_train, batch_size = 1, epochs = 5)

In [ ]:
# create testing dataset
test_data = scaled_data[training_data_len-60: , :]
#create dataset x_test and y_test
x_test = []
y_test = dataset[training_data_len: , :]

for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    

In [ ]:
# Convert to numpy array
x_test = np.array(x_test)
# Reshape data for LSTM
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
 

In [ ]:
# Get model model's predicted values

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)


In [ ]:
# Model Evaluation by RMSE
rmse = np.sqrt(np.mean(predictions-y_test)**2)
rmse 

In [ ]:
# Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['predictions'] = predictions

#Visualization

plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize = 18)
plt.ylabel('Close Price GBPUSD', fontsize = 18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'predictions']])
plt.legend(['Train','Val','predictions'], loc = 'lower right')
plt.show()

In [ ]:
# show valid and predicted
valid